In [121]:
include("StackyFan.jl")

coneRayDecomposition (generic function with 1 method)

In [122]:
"""

    BerghA(F::StackyFan,D::Array{Int64,1})

    Given a stacky fan F and a vector of booleans D representing the distinguished structure,
    returns a smooth stacky fan where the distinguished rays are independent.

    
"""

function BerghA(F::StackyFan,D::Array{Int64,1})
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(X.fan.INPUT_RAYS))
    coneList=getCones(X.fan)
    dim=size(rayMatrix,2)
    numRays=size(rayMatrix,1)
    #check if the vector D has length equal to the number of rays in F
    if numRays != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0: initialization
    while(true)
        rayMatrix=convert(Array{Int64,2},Array(X.fan.INPUT_RAYS))
        trueMatrix=convert(Array{Int64,2},Array(X.fan.RAYS))
        printn(rayMatrix)
        printn(trueMatrix)
        coneList=getCones(X.fan)
        printn(coneList)
        printn(" ")
        #A1: check if finished
        # Find S the set of cones that contain a distinguised ray and an interior a lattice point 
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndIntPoint(cone,rayMatrix,D),coneList)
        # If S is empty, the program terminates.
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            C=rowMinors(rayMatrix,cone)
            coneIntPoints=interiorPoints(C)
            if coneIntPoints==nothing
                # does the program ever reach this line if it breaks at S empty?
                return C
            end
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=Array{Int64,1}[]
        for (point,cone) in intPoints
            stackyPoint=coneRayDecomposition(cone,rayMatrix,point,X.scalars)
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        psi=minimalByLex(P)
        
        #A3 - perform root construction
        X=rootConstructionDistinguishedIndices(X,D,psi)
        
        #A3 - modify psi with respect to root construction
        for i in 1:length(psi)
            if D[i]==1 && psi[i]>0
                psi[i]=1
            end
        end
        
        while(count(x->x>0,psi)>1)
            #A4 - perform stacky star subdivision
            # Get the indices of the non-zero coefficients in psi
            supportCone=findall(x->x!=0,psi)
            X=stackyBlowup(X,[x-1 for x in supportCone],findBarycenter(supportCone,X.fan))
            D=push!(D,1)
        
            #A4 - modify psi
            newpsi=zeros(Int64,length(psi)+1)
            for i in 1:length(psi)
                if psi[i]>1
                    newpsi[i]=psi[i]-1
                end
            end
            newpsi[end]=1
            psi=newpsi
        end
    end
    return X
end

BerghA (generic function with 1 method)

In [129]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 2;1 0],INPUT_CONES=[[0,1]])

type: NormalToricVariety

INPUT_CONES
	{0 1}

INPUT_RAYS
	1 2
	1 0

In [130]:
F=addStackStructure(X,[3,4])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x000000002110c400), [3, 4], Dict("1,2" => 3, "1,0" => 4))

In [131]:
BerghA(F,[1,1])

[1 2; 1 0]
[1 2; 1 0]
Any[[1, 2], [1], [2]]
 
[1 2; 1 0; 2 2]
[1 2; 1 1; 1 0]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2]
[1 0; 1 1; 1 2]
Any[[1, 2], [2, 3], [1], [2], [3]]
 
[1 2; 1 0; 2 2; 2 2; 2 2; 2 2; 2 2; 2 2

┌ Warning: Interrupting polymake is not safe.
│ SIGINT is disabled while waiting for polymake to finish its computations.
└ @ Polymake /home/kitty/.julia/packages/Polymake/6iwMj/src/perlobj.jl:47


LoadError: InterruptException:

In [45]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[2 1 1;5 0 1;1 5 2],INPUT_CONES=[[0,1],[0,2]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{0 2}

INPUT_RAYS
	2 1 1
	5 0 1
	1 5 2